In [1]:
from pathlib import Path
import sys
import os
from pydantic import SecretStr

current_folder = Path.cwd()
src_root = current_folder.parent
project_root = src_root.parent
sys.path.insert(0, str(project_root))


from src.config import QDRANT_HOST, GEMINI_API_KEY

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

print("Imports successful!")

Imports successful!


### Configuration

In [2]:
COLLECTION_NAME = "dekkingen"
EMBEDDING_MODEL_NAME = "models/embedding-001"

embeddings = GoogleGenerativeAIEmbeddings(
            model=EMBEDDING_MODEL_NAME,
            google_api_key=SecretStr(GEMINI_API_KEY) if GEMINI_API_KEY else None
        )

### Retrieving

In [3]:
client = QdrantClient(host=QDRANT_HOST)
vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)

In [ ]:
query = "ik heb nierdialyse"

dense_result = vector_store.similarity_search_with_score(
    query,
    k=5,
)

hybrid_result = vector_store.hybrid_search_with_score(

In [6]:
for (doc, score) in result:
    print(doc.metadata)
    print(doc.page_content)
    print(f"Score: {score}\n")

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T19:48:53.194941', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Poliklinisch', 'header_5': '(Thuis)dialyse', '_id': 'e4ec27b5-9ac1-4eb6-8144-c9598168fc2b', '_collection_name': 'dekkingen'}
##### (Thuis)dialyse  
Dit verwijst naar de vergoeding van nierdialysebehandelingen, zowel thuis als in een kliniek. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.
Score: 0.82978415

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T19:48:53.194941', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 

In [10]:
for (doc, score) in result:
    print(doc.metadata)
    print(doc.page_content)
    print(f"Score: {score}\n")
    

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T16:08:12.871179', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Zwangerschap en bevalling', 'header_5': 'Sterilisatie', '_id': '50efb225-8aab-4cc3-bb7c-5d034896a868', '_collection_name': 'dekkingen'}
##### Sterilisatie  
Dit betreft de vergoeding voor een medische ingreep die resulteert in permanente onvruchtbaarheid. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.
Score: 0.8250388

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T16:08:12.871179', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md',